In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import os
import json

In [8]:
def load_tomi_dataset(
    data_dir="data/tom/ToMi",
    train_file="train.txt",
    trace_file="train.trace"
):
    # Read trace file to get belief labels
    trace_path = os.path.join(data_dir, trace_file)
    with open(trace_path, "r") as f:
        trace_lines = [line.strip() for line in f if line.strip()]
    # Read train file to get stories, questions, answers
    train_path = os.path.join(data_dir, train_file)
    with open(train_path, "r") as f:
        train_lines = [line.strip() for line in f if line.strip()]
    # Each story is separated by a blank line or by the repeated pattern
    # We'll parse by looking for lines starting with '1 ' (start of story)
    dataset = []
    i = 0
    trace_idx = 0
    while i < len(train_lines):
        # Collect story lines
        story_lines = []
        while i < len(train_lines) and not train_lines[i].startswith("9 ") and not train_lines[i].startswith("10 ") and not train_lines[i].startswith("11 "):
            story_lines.append(train_lines[i])
            i += 1
        # The question line
        if i < len(train_lines):
            question_line = train_lines[i]
            i += 1
        else:
            break
        # Parse question, answer
        if "\t" in question_line:
            q, a, _ = question_line.split("\t")
        else:
            q, a = question_line, ""
        # Get the corresponding belief label from trace file
        if trace_idx < len(trace_lines):
            trace = trace_lines[trace_idx]
            trace_idx += 1
            # The last field is the belief label
            belief = trace.split(",")[-1]
        else:
            belief = None
        # Save the entry
        dataset.append({
            "story": "\n".join(story_lines),
            "question": q,
            "answer": a,
            "belief": belief
        })
    return dataset

In [1]:
def load_bigtom_dataset():
    pass

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    output_hidden_states=True,
    output_attentions=True,
).to(device)
model.eval()
tokenizer.save_pretrained("./mistral-7b-tokenizer/")
model.save_pretrained("./mistral-7b-model/")

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

KeyboardInterrupt: 